<a href="https://colab.research.google.com/github/bharatbajoria/Summer-Internship/blob/master/Topic_Modelling_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PDF Reading

In [1]:
pip install pyPDF2

     |████████████████████████████████| 81kB 2.3MB/s 
  Created wheel for pyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=640ddea467e4c1e5655f2f279376744ad31cdf0bc3476b4888de7477839af680
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built pyPDF2


In [0]:
import PyPDF2

In [0]:
#Taking file input
#number of files
nf=2

file= open('Apr-18.PDF','rb')
file1= open('Apr-17.PDF','rb')
  

In [0]:
#A list of files
file_list=[file,file1]

In [0]:
# Reading number of pages
pages=[]

for i in range(nf):
  pdfReader = PyPDF2.PdfFileReader(file_list[i])
  n1=pdfReader.numPages
  pages.append(n1)

In [0]:
# A list containing all files, All_Docs[i] is i-th file 
# All_Docs[i][j] is j-th page of i-th file
All_Docs=[] 

In [0]:

for j in range(nf):
  pdfReader = PyPDF2.PdfFileReader(file_list[j])
  file_text=[] #takes string of page text
  for i in range(pages[j]):
    pageObj = pdfReader.getPage(i)
    a=pageObj.extractText()
    file_text.append(a)
  
  All_Docs.append(file_text)

In [12]:
type(All_Docs[0][1]),type(All_Docs[0])

(str, list)

# Finding Optimal number of Clusters

In [0]:
Dup_All_Docs=All_Docs[:]

In [14]:
type(Dup_All_Docs[0][0])

str

In [0]:
import nltk
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score

import pandas as pd
from nltk.tokenize import RegexpTokenizer
#from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from nltk.corpus import stopwords
import gensim

from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
tokenizer = RegexpTokenizer(r'\w+')
en_stop = set(stopwords.words('english'))
en_stop.add('the')#The is not in stopwords

In [0]:
tokens=[]


for j in All_Docs:
  x=[]
  for i in j:
    token=tokenizer.tokenize(i)
    token=[i for i in token if(not(str(i).isdigit() or not(str(i).isalpha())) and len(str(i)) > 2 )]
    token=[i.lower() for i in token if( i not in en_stop)]
    x.extend(token)
  
  tokens.extend(x)

In [21]:
len(tokens),tokens[:10]

(7941,
 ['press',
  'release',
  'department',
  'communication',
  'central',
  'office',
  'marg',
  'mumbai',
  'phone',
  'fax'])

In [0]:
#df9['Cleaned_PaperText'] = pd.Series(tokens)
        
vectorizer = TfidfVectorizer()

Y = vectorizer.fit_transform(tokens)

In [0]:
#Not Working

distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(Y)
    kmeanModel.fit(Y)
   # distortions.append(sum(np.min(cdist(Y, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

In [24]:
#2nd Method using Silhouette Score

n=0
ss=0
for n_cluster in range(2, 11):
    kmeans = KMeans(n_clusters=n_cluster).fit(Y)
    label = kmeans.labels_
    sil_coeff = silhouette_score(Y, label, metric='euclidean')  
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))
    if sil_coeff>ss:
      ss=sil_coeff
      n=n_cluster

For n_clusters=2, The Silhouette Coefficient is 0.008769778510606204
For n_clusters=3, The Silhouette Coefficient is 0.02675906514007355
For n_clusters=4, The Silhouette Coefficient is 0.0424980264635402
For n_clusters=5, The Silhouette Coefficient is 0.03969164604658748
For n_clusters=6, The Silhouette Coefficient is 0.03831576067188242
For n_clusters=7, The Silhouette Coefficient is 0.05342562085947669
For n_clusters=8, The Silhouette Coefficient is 0.07125054724095663
For n_clusters=9, The Silhouette Coefficient is 0.07540349676354198
For n_clusters=10, The Silhouette Coefficient is 0.07450058636998781


In [25]:
print("Optimal numbers of clusters are : {}".format(n))

Optimal numbers of clusters are : 9


# LDA

In [0]:

import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from gensim.corpora import Dictionary
from gensim.models import TfidfModel,LdaModel
from gensim.models.wrappers import LdaMallet

In [27]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:

dictionary=Dictionary([[i] for i in tokens])

In [29]:
print(dictionary)

Dictionary(2065 unique tokens: ['press', 'release', 'department', 'communication', 'central']...)


In [30]:
X=[dictionary.doc2bow([sents]) for sents in tokens]#getting bag of words.
tfidf=TfidfModel(X)

print(type(tfidf))
X=tfidf[X]


<class 'gensim.models.tfidfmodel.TfidfModel'>


In [0]:
from gensim.models import LdaMulticore,CoherenceModel

lda=LdaMulticore(X,num_topics=n,id2word=dictionary,workers=8)
lda1=LdaModel(X,num_topics=n,id2word=dictionary,update_every=5)

#lda.print_topics()

In [32]:

ldatopics = [[(word,prob) for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]
lda.show_topics(formatted=True)

[(0,
  '0.063*"prices" + 0.020*"march" + 0.019*"expected" + 0.017*"also" + 0.015*"capital" + 0.014*"increase" + 0.013*"impact" + 0.012*"inflation" + 0.012*"policy" + 0.012*"month"'),
 (1,
  '0.026*"outlook" + 0.024*"risks" + 0.021*"growth" + 0.018*"production" + 0.017*"policy" + 0.014*"due" + 0.013*"activity" + 0.012*"march" + 0.011*"global" + 0.010*"april"'),
 (2,
  '0.026*"growth" + 0.021*"effects" + 0.020*"per" + 0.015*"output" + 0.013*"market" + 0.012*"excluding" + 0.012*"resolution" + 0.012*"rising" + 0.011*"liquidity" + 0.011*"trade"'),
 (3,
  '0.044*"february" + 0.031*"policy" + 0.014*"would" + 0.013*"rate" + 0.013*"headline" + 0.012*"risks" + 0.011*"expectations" + 0.010*"activity" + 0.009*"likely" + 0.009*"imports"'),
 (4,
  '0.030*"year" + 0.029*"inflation" + 0.019*"monetary" + 0.018*"growth" + 0.016*"rates" + 0.016*"mpc" + 0.016*"may" + 0.015*"fiscal" + 0.012*"bank" + 0.012*"likely"'),
 (5,
  '0.065*"the" + 0.026*"rate" + 0.020*"meeting" + 0.019*"second" + 0.016*"per" + 0.01

In [0]:

ldatopics_word = [[word for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]

index=["Most Common Word"]
col=[]
a='Topic-'
c=''
for i in range(n):
  c+=a+str(i+1)
  col.append(c)
  c=''

len(col),len(ldatopics_word)

#topic_df=pd.DataFrame(data=ldatopics_word,columns=col)
topic_df=pd.DataFrame(data=ldatopics_word,index=col)

In [41]:
topic_df

,0,1,2,3,4,5,6,7,8,9
Topic-1,prices,march,expected,also,capital,increase,impact,inflation,policy,month
Topic-2,outlook,risks,growth,production,policy,due,activity,march,global,april
Topic-3,growth,effects,per,output,market,excluding,resolution,rising,liquidity,trade
Topic-4,february,policy,would,rate,headline,risks,expectations,activity,likely,imports
Topic-5,year,inflation,monetary,growth,rates,mpc,may,fiscal,bank,likely
Topic-6,the,rate,meeting,second,per,well,trade,high,inflation,also
Topic-7,january,committee,several,also,fuel,outlook,medium,investment,remains,higher
Topic-8,cent,reserve,impact,manufacturing,hra,headline,demand,consumer,although,survey
Topic-9,inflation,per,cent,food,economy,bank,march,exports,oil,could


In [0]:
topic_df_T=topic_df.T

In [74]:
topic_df_T

,Topic-1,Topic-2,Topic-3,Topic-4,Topic-5,Topic-6,Topic-7,Topic-8,Topic-9
0,economic,year,inflation,prices,growth,per,cent,policy,inflation
1,inflation,effects,headline,the,the,risks,prices,rate,reserve
2,economy,impact,february,oil,bank,cent,activity,also,outlook
3,committee,second,rising,inflation,february,fuel,high,march,expected
4,hra,survey,may,february,impact,monetary,basis,the,per
5,bank,last,several,production,food,january,account,recent,output
6,outlook,global,trade,well,financial,around,resolution,april,cpi
7,indian,likely,rates,months,various,exports,year,market,month
8,global,centre,food,estimates,remained,however,consumption,sector,mpc
9,vote,domestic,ahead,per,average,investment,february,central,services


In [76]:
len(topic_df_T["Topic-1"])

10

**Adding New File**

In [0]:

new_file=open('Apr-20.PDF','rb')
pdfReader = PyPDF2.PdfFileReader(new_file)
n1=pdfReader.numPages

new_file_pg=[]
for i in range(n1):
    pageObj = pdfReader.getPage(i)
    a=pageObj.extractText()
    new_file_pg.append(a)

#new_file_pg[:3]

In [0]:
new_token=[]
for i in new_file_pg:
    token=tokenizer.tokenize(i)
    token=[i for i in token if(not(str(i).isdigit() or not(str(i).isalpha())) and len(str(i)) > 2 )]
    token=[i.lower() for i in token if( i not in en_stop)]
    x.extend(token)
  
    new_token.extend(x)


Z=[dictionary.doc2bow([sents]) for sents in new_token]

In [0]:
newlda=lda1[Z]


In [45]:
len(Z)

83752

In [0]:
new_topic=[]
for topic in newlda:
  #new_topic.append(prob)
  new_topic.append(topic)

In [150]:
len(new_topic),type(new_topic[0])#,new_topic[:2]

(83752, list)

In [48]:
lda1.get_document_topics(Z,per_word_topics=True)

# Evaluating Model: Topic Wise


In [49]:
tk=[[i] for i in tokens]
ldatopics = [[word for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]
lda_coherence = CoherenceModel(topics=ldatopics, texts=tk, dictionary=dictionary, window_size=10).get_coherence()
lda_coherence

0.7186240625449318

In [0]:
coherence_score=[]
tk=[[i] for i in tokens]
for i in range(n):
  lda=LdaMulticore(X,num_topics=n,id2word=dictionary,workers=8)
  ldatopics = [[word for word, prob in topic] for topicid, topic in lda.show_topics(formatted=False)]
  lda_coherence = CoherenceModel(topics=ldatopics, texts=tk, dictionary=dictionary, window_size=10).get_coherence()
  coherence_score.append(lda_coherence)



In [51]:
print("Max Score is: {} for {} clusters".format(max(coherence_score),coherence_score[coherence_score.index(max(coherence_score))]))

Max Score is: 0.7193956145444242 for 0.7193956145444242 clusters


In [52]:
print("Max Score is: {} for {} clusters".format(max(coherence_score),coherence_score.index(max(coherence_score))))

Max Score is: 0.7193956145444242 for 6 clusters


# Evaluating Words in a Topic : Cosine scores.

In [0]:
Dup_All_Docs=All_Docs[0]+All_Docs[1]
df=[]

In [0]:
for j in All_Docs:
  for i in j:
    df.append(i)


In [55]:
len(All_Docs[0])

14

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectoriser = CountVectorizer().fit(Dup_All_Docs)
X = vectoriser.transform(Dup_All_Docs)
df_vec=pd.DataFrame(X.A, columns=sorted(vectoriser.vocabulary_.keys(), key=lambda k: vectoriser.vocabulary_[k]))

In [0]:
df_page=pd.DataFrame(data=Dup_All_Docs)
#df_page.head()

In [63]:
type(df),len(df)

(list, 26)

In [0]:
word_data=[]

for i in df:
    token=tokenizer.tokenize(i)
    token=[i for i in token if(not(str(i).isdigit() or not(str(i).isalpha())) and len(str(i)) > 2 )]
    token=[i.lower() for i in token if( i not in en_stop)]
    word_data.append(token)

In [0]:
model1 = gensim.models.Word2Vec(word_data, min_count = 1,size = 100, window = 5) 

In [77]:
topic_df_T

,Topic-1,Topic-2,Topic-3,Topic-4,Topic-5,Topic-6,Topic-7,Topic-8,Topic-9
0,economic,year,inflation,prices,growth,per,cent,policy,inflation
1,inflation,effects,headline,the,the,risks,prices,rate,reserve
2,economy,impact,february,oil,bank,cent,activity,also,outlook
3,committee,second,rising,inflation,february,fuel,high,march,expected
4,hra,survey,may,february,impact,monetary,basis,the,per
5,bank,last,several,production,food,january,account,recent,output
6,outlook,global,trade,well,financial,around,resolution,april,cpi
7,indian,likely,rates,months,various,exports,year,market,month
8,global,centre,food,estimates,remained,however,consumption,sector,mpc
9,vote,domestic,ahead,per,average,investment,february,central,services


In [0]:
#ds2

Cosine Similarity Score

In [0]:
# List of topic wise words
ds=[]
for i in range(len(col)):
  ds.append(list(topic_df_T[col[i]]))

ds# ds[i] is i-th list of Topic-i

simscore_topic=[]#initiating mtx for sim scores/cosine scores


for p in range(len(ds)):
  k=len(ds[p])
  o=0

  sim=[]
  simt=[]
  
  for i in range(k):
    simt=[]
    for j in range(k)  :
      o=model1.similarity(ds[p][i],ds[p][j])
      simt.append(o)

    sim.append(simt)

  simscore_topic.append(sim)
# 3-D list, simscore_topic[i] is similarity scores for topic-i
# simscore_topic[i][j] is list of similarity score of j-th word in i-th Topic with other words




for j in range(len(ds)):
  for i in range(k-1):
    print(ds[j][i])
    print(simscore_topic[j][i])
    #print(model1.similarity(topic1[i],topic1[i+1]))
   

In [146]:
gbl = globals()


#df_topic_name = ['SymbolA','SymbolB', 'SymbolC' .... 'SymbolN']

for i in range(len(col)):   

   gbl[str('df')+col[i]] = pd.DataFrame(simscore_topic[i],index=ds[i],columns=ds[i])
   print("Similarity Scores matrix for Topic-{}".format(i+1))
   print(gbl[str('df')+col[i]])
   print("\n\n")

Similarity Scores matrix for Topic-1
           economic  inflation   economy  ...    indian    global      vote
economic   1.000000   0.596814  0.475185  ...  0.339595  0.484141  0.197012
inflation  0.596814   1.000000  0.590754  ...  0.559473  0.692162  0.314516
economy    0.475185   0.590754  1.000000  ...  0.399332  0.468677  0.321382
committee  0.447275   0.551851  0.386181  ...  0.242994  0.499354  0.229896
hra        0.350965   0.537150  0.427196  ...  0.212327  0.489798  0.208487
bank       0.493649   0.744196  0.532851  ...  0.507422  0.585065  0.312950
outlook    0.469598   0.578973  0.491763  ...  0.433137  0.435138  0.306757
indian     0.339595   0.559473  0.399332  ...  1.000000  0.493038  0.284840
global     0.484141   0.692162  0.468677  ...  0.493038  1.000000  0.306258
vote       0.197012   0.314516  0.321382  ...  0.284840  0.306258  1.000000

[10 rows x 10 columns]



Similarity Scores matrix for Topic-2
              year   effects    impact  ...    likely    centre

In [139]:
#Topic1_df=pd.DataFrame(simscore_topic[0],index=ds[0],columns=ds[0])
gbl[dfTopic-1]

NameError: ignored